# Homework 4

# Задача №1 - Лего детальки

Имеется 2 выборки из датасета фотографий лего картинок. Структура одной выборки следующая:
```
test
| -- 'target.jpg'
| -- source
|    | -- '*.jpg'
|           ...
|    | -- '*.jpg'
```
В выборке есть опорное изображение - target.jpg и набор из 20 картинок в папке source.

Ваша задача - написать программу, которая прочитает все изображения и определит на каких изображениях из папки source сфотографирована такая же лего деталька, что и деталь в target.jpg

Результатом выполнения программы должен быть файлик с названиями подходящих изображений. Пример в Datasets/gt
Снизу представлены 2 вспомогательные функции: чтение всех изображений из папки и посчет получившихся метрик точности

Результат выполнения данной задачи напрямую зависит от полученных метрик. Полученные метрики будут вычисляться из метрик по двум тестам $Accuracy=min({Accuracy}_{test1}, {Accuracy}_{test2}$)

| Accuracy | Recall | % Выполнения |
|:--------:|:------:|:------------:|
|< 0.6|< 0.4|0%|
|  >= 0.6  | >= 0.4 |     75%      |
|  >= 0.7  | >= 0.5 |     100%     |
|  >= 0.9  | >= 0.8 |     150%     |

<table><tr>
    <td> <img src="Datasets/test1/target.jpg" alt="Drawing" style="width: 400px;"/> </td>
      <td> <img src="Datasets/test2/target.jpg" alt="Drawing" style="width: 400px;"/> </td>

</tr></table>

In [ ]:
from pathlib import Path
import cv2
import numpy as np
import os

def read_all_image_paths(folder):
    folder = Path(folder)
    image_paths = [i.__str__() for i in folder.glob('*.jpg')]
    return image_paths

def compare_images(target_img, source_img):
    """Сравнивает два изображения с помощью SIFT и FLANN"""
    if target_img is None or source_img is None:
        return 0.0
    
    # Инициализация детектора SIFT
    sift = cv2.SIFT_create()
    
    # Ищем ключевые точки и дескрипторы
    kp1, des1 = sift.detectAndCompute(target_img, None)
    kp2, des2 = sift.detectAndCompute(source_img, None)
    
    if des1 is None or des2 is None or len(kp1) == 0 or len(kp2) == 0:
        return 0.0
    
    # Используем FLANN matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    threshold = 0.7
    good_matches = []
    for m, n in matches:
        if m.distance < threshold * n.distance:
            good_matches.append(m)
    
    # Оценка похожести
    similar = len(good_matches) / min(len(kp1), len(kp2))
    return similar

def find_matching_images(test_folder, threshold=0.15):
    """Находит изображения, соответствующие target.jpg"""
    test_folder = Path(test_folder)    
    
    # Читаем target изображение
    target_path = test_folder / 'target.jpg'    
    
    target_img = cv2.imread(str(target_path), cv2.IMREAD_COLOR)    
    
    # Читаем все source изображения
    source_folder = test_folder / 'source'    
    
    image_paths = read_all_image_paths(source_folder)
    matches = []
    
    for img_path in image_paths:
        source_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if source_img is None:
            continue
            
        similarity = compare_images(target_img, source_img)
        img_name = os.path.basename(img_path)
        if similarity >= threshold:
            matches.append(img_name)
    
    return matches

def save_results(matches, output_file):
    """Сохраняет результаты в файл"""
    with open(output_file, 'w') as f:
        for match in matches:
            f.write(f"{match}\n")


def return_metrics(gt_path, prediction_path):
    with open(gt_path) as f:
        gt = set([i.strip('\n') for i in f.readlines() if len(i)])
    with open(prediction_path) as f:
        pred = set([i.strip('\n') for i in f.readlines() if len(i)])

    len_dataset = 20

    TP, TN, FP, FN = (0, ) * 4
    for i in pred:
        if i in gt:
            TP += 1
            gt.remove(i)
        else:
            FP += 1
    FN = len(gt)
    TN = len_dataset - TP - FP - FN

    accuracy = (TP + TN) / len_dataset
    recall = TP / (TP + FN)

    print(f'Acc: {accuracy:.2f}')
    print(f'Recall: {recall:.2f}')

print('All paths for test1:')
images_folder = Path(os.path.abspath('')) / 'Datasets' / 'test1' / 'source'
print(read_all_image_paths(images_folder))

print('Example of calculating metrics:')
gt_test1 = Path(os.path.abspath('')) / 'Datasets' / 'gt' / 'test1.txt'
pred = Path(os.path.abspath('')) / 'test1.txt'

return_metrics(gt_test1, pred)

All paths for test1:
['c:\\Users\\matve\\Desktop\\Programing_4Semester\\Computer_Vision\\Homework_CV\\homework_4\\Datasets\\test1\\source\\00000_2.1_01249.jpg', 'c:\\Users\\matve\\Desktop\\Programing_4Semester\\Computer_Vision\\Homework_CV\\homework_4\\Datasets\\test1\\source\\00001_1.2_00742.jpg', 'c:\\Users\\matve\\Desktop\\Programing_4Semester\\Computer_Vision\\Homework_CV\\homework_4\\Datasets\\test1\\source\\00002_1.1_00798.jpg', 'c:\\Users\\matve\\Desktop\\Programing_4Semester\\Computer_Vision\\Homework_CV\\homework_4\\Datasets\\test1\\source\\00002_2.2_00703.jpg', 'c:\\Users\\matve\\Desktop\\Programing_4Semester\\Computer_Vision\\Homework_CV\\homework_4\\Datasets\\test1\\source\\00003_2.1_00721.jpg', 'c:\\Users\\matve\\Desktop\\Programing_4Semester\\Computer_Vision\\Homework_CV\\homework_4\\Datasets\\test1\\source\\00003_2.2_00345.jpg', 'c:\\Users\\matve\\Desktop\\Programing_4Semester\\Computer_Vision\\Homework_CV\\homework_4\\Datasets\\test1\\source\\00003_2.2_01170.jpg', 'c:\\

# Задача №2 - Найти шаблон на изображении с помощью HOG и кросс-корреляции

Ваша задача - написать программу, которая сможет найти шаблон одного глаза на нескольких изображениях, в том числе затемненных. Использовать нужно HOG представление изображений, поиск шаблона сделать с помощью кросс-корреляции 

Изображение глаз человека и шаблон, который нужно на нем найти, находятся в папке hog

<table><tr>
    <td> <img src="hog/eye.png" alt="Drawing" style="width: 400px;"/> 
</tr></table>     

<table><tr>
    <td> <img src="hog/eyes.jpg" alt="Drawing" style="width: 400px;"/> </td>
      <td> <img src="hog/eyes_dark.jpg" alt="Drawing" style="width: 400px;"/> </td>
      <td> <img src="hog/woman.jpg" alt="Drawing" style="width: 200px;"/> </td>

</tr></table>

In [10]:
# Ваш код